In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stderr
from io import StringIO
from pathlib import Path

from QligFEP.CLI.lomap_wrap_cli import LomapWrap

ligfiles = sorted(Path.cwd().glob("*/ligands/ligands.sdf"))

ModuleNotFoundError: No module named 'konnektor'

In [ ]:
from pathlib import Path

import kartograf
import openfe
from kartograf.filters import (
    filter_ringbreak_changes,
    filter_ringsize_changes,
    filter_whole_rings_only,
)
from lomap.graphgen import GraphGen
from rdkit import Chem
import gufe
from gufe import (
    LigandAtomMapping,
    AtomMapper,
    LigandNetwork,
)
from loguru import logger
import numpy as np
import itertools
import networkx as nx
from typing import Optional, Union, Callable


# modified from openfe.ligand_network_planning.
def generate_lomap_network(
    molecules: list[gufe.SmallMoleculeComponent],
    mappers: Union[AtomMapper, list[AtomMapper]],
    scorer: Callable,
    distance_cutoff: float = 0.4,
    max_path_length=6,
    actives: Optional[list[bool]] = None,
    max_dist_from_active=2,
    require_cycle_covering: bool = True,
    radial: bool = False,
    fast: bool = False,
    hub: Optional[gufe.SmallMoleculeComponent] = None,
) -> LigandNetwork:
    """Generate a LigandNetwork according to Lomap's network creation rules

    Parameters
    ----------
    molecules : list[SmallMoleculeComponent]
       molecules to map
    mappers : list[AtomMapper] or AtomMapper
       one or more Mapper functions to use to propose edges
    scorer: function
       scoring function for edges.  Should be a function which takes an AtomMapping and returns a value from 1.0 (best)
       to 0.0 (worst).  These values are use as the "distance" between two molecules, and compared against the
       'distance_cutoff' parameter
    distance_cutoff : float
       the maximum distance/dissimilarity between two molecules for an edge to be accepted
    max_path_length : int
      maximum distance between any two molecules in the resulting network
    actives : list[bool]
      for each molecule, if it is tagged as an active molecule
    max_dist_from_active
      when 'actives' is given, constrains the resulting map to
    require_cycle_covering : bool
      add cycles into the network
    radial : bool
      construct a radial/starmap network.  Note that this the map will not necessarily be a true radial map; edges
      will still obey the distance_cutoff and if 'require_cycle_covering' is true, this radial map will still
      feature cycles, default False
    fast : bool
      hmmm...
    hub : SmallMoleculeComponent, optional
      if radial, force this ligand to be the centre/hub of the radial graph
    """
    if not mappers:
        raise ValueError("At least one Mapper must be provided")
    if isinstance(mappers, gufe.AtomMapper):
        mappers = [mappers]
    if actives is None:
        actives = [False] * len(molecules)

    # gen n x n mappings with scores
    # initially all zero scores, i.e. impossible
    mtx = np.zeros((len(molecules), len(molecules)), dtype=float)
    # np array of mappings
    mps = np.zeros_like(mtx, dtype=object)

    # for all combinations of molecules
    for i, j in itertools.combinations(range(len(molecules)), 2):
        mA, mB = molecules[i], molecules[j]

        # pick best score across all mappings from all mappings
        best_mp: Optional[LigandAtomMapping] = None
        best_score = 0.0
        for mapper in mappers:
            mp: LigandAtomMapping
            score: float
            try:
                score, mp = max(
                    (scorer(mp), mp) for mp in (mapper.suggest_mappings(mA, mB))
                )
            except ValueError:
                # if mapper returned no mappings
                continue
            else:
                if score > best_score:
                    best_mp = mp
                    best_score = score

        if best_mp is None:
            # logger.debug(f"Found no mapping for {mA} {mB}")
            continue

        # logger.debug(f"Mapping for {mA} {mB} has score {best_score}")

        mtx[i, j] = mtx[j, i] = best_score
        mps[i, j] = mps[j, i] = best_mp.with_annotations({"score": best_score})

    gg = GraphGen(
        score_matrix=mtx,
        ids=list(range(mtx.shape[0])),
        names=[m.name for m in molecules],
        max_path_length=max_path_length,
        actives=actives,
        max_dist_from_active=max_dist_from_active,
        similarity_cutoff=1 - distance_cutoff,
        require_cycle_covering=require_cycle_covering,
        radial=radial,
        fast=fast,
        hub=hub.name if hub else None,
    )
    n: nx.Graph = gg.resultGraph
    return n


def gen_ligand_network(smcs):
    """
    Creates the Lomap ligand network using the KartografAtomMapper and
    the Lomap scorer.

    Parameters
    ----------
    smcs : list[SmallMoleculeComponents]
      List of SmallMoleculeComponents of the ligands.

    Returns
    -------
    openfe.LigandNetwork
      The Lomap generated LigandNetwork.
    """
    print("INFO: Generating Lomap Network")
    mapping_filters = [
        filter_ringbreak_changes,  # default
        filter_ringsize_changes,  # default
        filter_whole_rings_only,  # default
    ]
    mapper = kartograf.KartografAtomMapper(
        atom_map_hydrogens=True,
        additional_mapping_filter_functions=mapping_filters,
    )
    scorer = openfe.lomap_scorers.default_lomap_score
    ligand_network = generate_lomap_network(
        molecules=smcs, mappers=mapper, scorer=scorer
    )
    return ligand_network


def run_inputs(ligands, output):
    """
    Generate run json files for RBFE calculations

    Parameters
    ----------
    ligands : pathlib.Path
      A Path to a ligands SDF.
    pdb : pathlib.Path
      A Path to a protein PDB file.
    cofactors : Optional[pathlib.Path]
      A Path to an SDF file containing the system's cofactors.
    output: pathlib.Path
      A Path to a directory where the transformation json files
      and ligand network graphml file will be stored into.
    """
    # Create the output directory -- default to alchemicalNetwork, fail if it exists
    output.mkdir(exist_ok=True, parents=True)

    # Create the small molecule components of the ligands
    rdmols = [mol for mol in Chem.SDMolSupplier(str(ligands), removeHs=False)]
    smcs = [openfe.SmallMoleculeComponent.from_rdkit(mol) for mol in rdmols]

    # Create ligand network
    ligand_network = gen_ligand_network(smcs)

    # # Store the ligand network as a graphml file
    # with open(output / "ligand_network.graphml", mode="w") as f:
    #     f.write(ligand_network.to_graphml())

    return ligand_network


targets = [
    "bace",
    "p38",
    "thrombin",
    "cdk2",
    "hif2a",
    "pfkfb3",
    "tnks2",
    "cdk8",
    "jnk1",
    "ptp1b",
    "shp2",
    "tyk2",
    "cmet",
    "syk",
    "eg5",
    "mcl1",
]


for target in targets:
    if target == "pfkfb3":
        print("Skipping PFKFB3 (not yet in the repo)")
        continue
    pass

target = "bace"
rootpath = Path().absolute()
network = run_inputs(
    ligands=rootpath / f"{target}/ligands/ligands.sdf",
    output=rootpath / f"{target}/ligands/network",
)

Skipping PFKFB3 (not yet in the repo)
INFO: Generating Lomap Network


In [20]:
network
result = {"nodes": [], "edges": []}
for node, data in network.nodes(data=True):
    node_dict = {"id": node}
    node_dict.update(data)
    result["nodes"].append(node_dict)

# Add edges to the result
for source, target, data in network.edges(data=True):
    edge_dict = {"source": source, "target": target}
    edge_dict.update(data)
    result["edges"].append(edge_dict)

new_nodes = [{"name": node["fname_comp"]} for node in result["nodes"]]

# Create a mapping of old IDs to new names
id_to_name = {node["id"]: node["fname_comp"] for node in result["nodes"]}

# Transform edges
new_edges = []
for edge in result["edges"]:
    new_edge = {
        "from": id_to_name[edge["source"]],
        "to": id_to_name[edge["target"]],
        "similarity": edge["similarity"],
        "strict_flag": edge["strict_flag"],
    }
    new_edges.append(new_edge)

# Create the new data structure
new_data = {"nodes": new_nodes, "edges": new_edges}

In [21]:
new_data

{'nodes': [{'name': 'CAT-13n'},
  {'name': 'CAT-13m'},
  {'name': 'CAT-13j'},
  {'name': 'CAT-4n'},
  {'name': 'CAT-4p'},
  {'name': 'CAT-4i'},
  {'name': 'CAT-13e'},
  {'name': 'CAT-4d'},
  {'name': 'CAT-4a'},
  {'name': 'CAT-4m'},
  {'name': 'CAT-4k'},
  {'name': 'CAT-4l'},
  {'name': 'CAT-4j'},
  {'name': 'CAT-4c'},
  {'name': 'CAT-13k'},
  {'name': 'CAT-4o'},
  {'name': 'CAT-4b'},
  {'name': 'CAT-17h'},
  {'name': 'CAT-17e'},
  {'name': 'CAT-17b'},
  {'name': 'CAT-13d'},
  {'name': 'CAT-13h'},
  {'name': 'CAT-17g'},
  {'name': 'CAT-17f'},
  {'name': 'CAT-24'},
  {'name': 'CAT-17i'},
  {'name': 'CAT-17a'},
  {'name': 'CAT-17c'},
  {'name': 'CAT-17d'},
  {'name': 'CAT-13a'},
  {'name': 'CAT-13i'},
  {'name': 'CAT-13c'},
  {'name': 'CAT-13b'},
  {'name': 'CAT-13o'},
  {'name': 'CAT-13g'},
  {'name': 'CAT-13f'}],
 'edges': [{'source': 'CAT-13n',
   'target': 'CAT-4p',
   'similarity': 0.8607079764250578,
   'strict_flag': True},
  {'source': 'CAT-13n',
   'target': 'CAT-13j',
   'simil

In [5]:
import networkx as nx

target = "bace"
rootpath = Path().absolute()


def graphml_to_dict(file_path):
    # Read the GraphML file
    G = nx.read_graphml(file_path)

    # Initialize the result dictionary
    result = {"nodes": [], "edges": []}

    # Add nodes to the result
    for node, data in G.nodes(data=True):
        node_dict = {"id": node}
        node_dict.update(data)
        result["nodes"].append(node_dict)

    # Add edges to the result
    for source, target, data in G.edges(data=True):
        edge_dict = {"source": source, "target": target}
        edge_dict.update(data)
        result["edges"].append(edge_dict)

    return result


graphdict = graphml_to_dict(
    rootpath / f"{target}/ligands/network/ligand_network.graphml"
)

In [11]:
graphdict["nodes"][0]["moldict"]

'{":version:": 1, "__module__": "gufe.components.smallmoleculecomponent", "__qualname__": "SmallMoleculeComponent", "atoms": [[6, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [7, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, false, 1, 0, {"molParity": 1}], [6, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [7, 0, 1, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [7, 0, 0, false, 0, 0, {}], [7, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [6, 0, 0, false, 0, 0, {}], [8, 0, 0, false, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [6, 0, 0, true, 0, 0, {}], [1, 0, 0, false, 0, 0, {}], [1, 0, 0, false, 0, 0, {}], [1, 0, 0, false, 0, 0, {}], [1, 0, 0, false, 0, 0, {}], [1, 0, 0, false, 0, 0, {}], [1, 

In [3]:
# # Create a string IO object to capture the output
# output = StringIO()

# with redirect_stderr(output):  # suppress output from lomap...
#     for ligs in ligfiles:
#         try:
#             lomap_wrapper = LomapWrap(str(ligs))
#         except FileExistsError:  # this in case the temp directory already exists
#             lomap_wrapper = LomapWrap(str(ligs.parent / "ligands"))
#         lomap_wrapper.run_lomap()

In [4]:
import argparse
from QligFEP.CLI.qparams_cli import main
from tqdm.notebook import tqdm

for ligs in tqdm(ligfiles, desc="Processing ligands"):
    withH_mols = ligs
    # if len(list(ligs.parent.glob("*.prm"))) > 0:
    #     print(f"Skipping {withH_mols}")
    #     continue  # skip if already processed
    print(f"Processing {withH_mols}")
    args = argparse.Namespace()
    args.input = withH_mols
    args.FF = "OpenFF"
    args.nagl = False
    args.log = "debug"
    args.parallel = 8
    main(args)

Processing ligands:   0%|          | 0/15 [00:00<?, ?it/s]

Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/cdk2/ligands/ligands.sdf


2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 20. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 1oi9. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 26. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 1oiu. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 1h1q. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 21. Reindexed.
2024-09-09 14:12:52 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 22. Rein

Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/cdk8/ligands/ligands.sdf


2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 40. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 30. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 42. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 37. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 38. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 13. Reindexed.
2024-09-09 14:19:30 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule 41. Reindexed.

...

2024-09-09 20:06:52 | DEBUG    | QligFEP.openff2Q:_set_forcefield:64 - Forcefield for the ligand parameters: openff-2.2.0.offxml
2024-09-09 20:06:54 | INFO     | QligFEP.openff2Q:process_ligands:106 - Calculating charges

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [02:32<00:00,  6.93s/it]
2024-09-09 20:21:32 | INFO     | QligFEP.openff2Q:process_ligands:111 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-09-09 20:21:32 | DEBUG    | QligFEP.openff2Q:process_ligands:112 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/thrombin/ligands
2024-09-09 20:21:33 | WARNING  | QligFEP.openff2Q:process_ligands:125 - Formal charges of ligands in .sdf are not unique: {'4': '2.000', '6c': '1.000', '1a': '1.000', '1b': '1.000', '6d': '1.000', '2e': '1.000', '3a': '1.000', '1d': '1.000', '1c': '1.000', '2d': '1.000', '2b': '1.000', '

Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/ligands/ligands.sdf


2024-09-09 20:21:33 | DEBUG    | QligFEP.openff2Q:_set_forcefield:64 - Forcefield for the ligand parameters: openff-2.2.0.offxml
2024-09-09 20:21:35 | INFO     | QligFEP.openff2Q:process_ligands:106 - Calculating charges

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [03:56<00:00,  8.76s/it]
2024-09-09 20:29:50 | INFO     | QligFEP.openff2Q:process_ligands:111 - Done! Writing .lib, .prm and .pdb files for each ligand
2024-09-09 20:29:50 | DEBUG    | QligFEP.openff2Q:process_ligands:112 - Output path: /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/ligands
2024-09-09 20:29:50 | WARNING  | QligFEP.openff2Q:process_ligands:125 - Formal charges of ligands in .sdf are not unique: {'8c': '1.000', '8d': '1.000', '5f': '0.000', '5o': '0.000', '8e': '1.000', '8b': '1.000', '8f': '1.000', '5i': '0.000', '5g': '0.000', '8a': '1.000', '5k': '0.000', '3a

Processing /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tyk2/ligands/ligands.sdf


2024-09-09 20:29:50 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule ejm_45. Reindexed.
2024-09-09 20:29:50 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule jmc_28. Reindexed.
2024-09-09 20:29:50 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule ejm_55. Reindexed.
2024-09-09 20:29:50 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule ejm_43. Reindexed.
2024-09-09 20:29:50 | WARNING  | QligFEP.chemIO:_force_H_reindexing:51 - Hydrogens not at the end of the atom list for molecule ejm_48. Reindexed.
2024-09-09 20:29:50 | DEBUG    | QligFEP.openff2Q:_set_forcefield:64 - Forcefield for the ligand parameters: openff-2.2.0.offxml
2024-09-09 20:29:51 | INFO     | QligFEP.openff2Q:process_ligands:106 - Calculating charges

100%|██████████████████████████████████████